Import all relevant libraries

In [1]:
import os, sys
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import scipy.stats as stats
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import datetime as dt
from netCDF4 import Dataset, MFDataset
import h5py
import xarray as xr
from mpl_toolkits.mplot3d import axes3d
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib.colors as mcolors
import cftime
import matplotlib.lines as mlines
from sklearn.decomposition import PCA
from sklearn import preprocessing, cluster
from sklearn.linear_model import LinearRegression
from scipy.spatial.distance import cdist
import csv
from itertools import product
from sklearn.ensemble import RandomForestRegressor
import matplotlib.patches as patches
import statsmodels.api as sm

In [2]:
os.environ['HDF5_USE_FILE_LOCKING']='FALSE'

prefix = '/data/rong4/Data/'
prefix2 = '/data/rong3/achang1029/'

Lat/Lon Array at 0.25 degrees over -85 to -30 lon, 15 to -35 lat. Covers South America

In [3]:
vpd_lon_array_sa = np.arange(-85, -29.9, 0.25)
vpd_lat_array_sa = np.arange(15, -35.1, -0.25)

Load All Relevant Datasets

In [4]:
#Old method of calculating VPD directly using average
# t2_sa = Dataset(prefix2 + 'ERA5/daily/t2_daily_sa_1979-2023.nc').variables['t2m'][:]
d2_sa = Dataset(prefix2 + 'ERA5/daily/d2_daily_sa_1979-2023.nc').variables['d2m'][:]

In [5]:
#For consistency, calculate VPD the same way as done as in CMIP6 calculations
t2max_sa = Dataset(prefix2 + 'ERA5/daily/t2max_daily_sa_1979-2023.nc').variables['t2m'][:]
t2min_sa = Dataset(prefix2 + 'ERA5/daily/t2min_daily_sa_1979-2023.nc').variables['t2m'][:]
t2_sa = (t2max_sa + t2min_sa) / 2

In [6]:
#Precipitation data from CPC. Different resolution and bounds than ERA5. Below indices cover overall SA domain
precip_sa = Dataset(prefix2 + 'CPC/precip_1979-2023.nc').variables['__xarray_dataarray_variable__'][:,150:250,550:660] 

In [6]:
#Heatmap testing of Analog correlation:

# z250_sa = Dataset(prefix2 + 'ERA5/daily/z250_daily_sa_1979-2020.nc').variables['z'][:]
z500_sa = Dataset(prefix2 + 'ERA5/daily/z500_daily_sa_1979-2023.nc').variables['__xarray_dataarray_variable__'][:]
# z700_sa = Dataset(prefix2 + 'ERA5/daily/z700_daily_sa_1979-2020.nc').variables['z'][:]
# z850_sa = Dataset(prefix2 + 'ERA5/daily/z850_daily_sa_1979-2020.nc').variables['z'][:]

# u250_sa = Dataset(prefix2 + 'ERA5/daily/u250_daily_sa_1979-2020.nc').variables['u'][:]
# u500_sa = Dataset(prefix2 + 'ERA5/daily/u500_daily_sa_1979-2020.nc').variables['u'][:]
# u700_sa = Dataset(prefix2 + 'ERA5/daily/u700_daily_sa_1979-2020.nc').variables['u'][:]
# u850_sa = Dataset(prefix2 + 'ERA5/daily/u850_daily_sa_1979-2020.nc').variables['u'][:]

# v250_sa = Dataset(prefix2 + 'ERA5/daily/v250_daily_sa_1979-2020.nc').variables['v'][:]
# v500_sa = Dataset(prefix2 + 'ERA5/daily/v500_daily_sa_1979-2020.nc').variables['v'][:]
# v700_sa = Dataset(prefix2 + 'ERA5/daily/v700_daily_sa_1979-2020.nc').variables['v'][:]
# v850_sa = Dataset(prefix2 + 'ERA5/daily/v850_daily_sa_1979-2020.nc').variables['v'][:]

# r250_sa = Dataset(prefix2 + 'ERA5/daily/r250_daily_sa_1979-2020.nc').variables['r'][:]
r500_sa = Dataset(prefix2 + 'ERA5/daily/r500_daily_sa_1979-2023.nc').variables['__xarray_dataarray_variable__'][:]
# r700_sa = Dataset(prefix2 + 'ERA5/daily/r700_daily_sa_1979-2020.nc').variables['r'][:]
# r850_sa = Dataset(prefix2 + 'ERA5/daily/r850_daily_sa_1979-2020.nc').variables['r'][:]

VPD Calculation. Uses SVP function on T2M and D2M to get es and ea, then convert to VPD

In [6]:
def svp(x): #Temperature in K, svp in Pa
    return 610.8 * np.exp(17.27 * (x - 273.15) / (x - 35.85))

In [7]:
es_sa = (svp(t2max_sa) + svp(t2min_sa)) / 2
ea_sa = svp(d2_sa)

In [8]:
vpd_sa = es_sa - ea_sa

Cells to track memory usage within this notebook and debug and kernel crashes due to exceeding memory limits.

In [9]:
del t2_sa, d2_sa, t2max_sa, t2min_sa

In [41]:
#Run this cell to determine memory usage of variables
local_vars = list(locals().items())
for var, obj in local_vars:
    print(var, sys.getsizeof(obj))

__name__ 57
__doc__ 113
__package__ 16
__loader__ 16
__spec__ 16
__builtin__ 72
__builtins__ 72
_ih 472
_oh 360
_dh 64
In 472
Out 360
get_ipython 64
exit 48
quit 48
_ 32
__ 32
___ 32
_i 908
_ii 909
_iii 111
_i1 897
os 72
sys 72
plt 72
cm 72
np 72
stats 72
pd 72
ccrs 72
cfeature 72
dt 72
hp 72
ap 72
Dataset 408
MFDataset 1064
h5py 72
xr 72
axes3d 72
ListedColormap 1472
LinearSegmentedColormap 1472
mcolors 72
cftime 72
mlines 72
PCA 1064
preprocessing 72
cluster 72
LinearRegression 1064
cdist 136
csv 72
product 408
make_blobs 136
patches 72
sm 72
_i2 158
prefix 66
prefix2 72
_i3 142
vpd_lon_array_sa 1880
vpd_lat_array_sa 1720
_i4 1514
r700_sa 168
_i5 135
w700_sa 168
_i6 151
svp 136
_i7 106
_i8 521
predict_data_linear_regression 136
_i9 309
linear_regression 136
_i10 551
linear_regression_slope_map 136
_i11 218
t2_sa 168
d2_sa 168
_i12 106
es_daily_avg_sa 168
ea_daily_avg_sa 168
_i13 101
vpd_daily_avg_sa 168
_i14 920
plot_data 136
_i15 682
plot_clusters 136
_i16 967
plot_data_anom 136
_i1

Relevant plotting functions

In [10]:
#Function to plot data
def plot_data(dm, ln, lt, clim, extent=[-180, 180, -90, 90], cmap='viridis', title=''):
    ds = dm.copy()
    fig = plt.figure(figsize=(10,10))
    ax = plt.axes(projection=ccrs.PlateCarree()) 
    cs = plt.contourf(ln, lt, ds, levels=np.linspace(clim[0], clim[1], 100), cmap=cmap, extend='both')
    ax.set_xticks(np.arange(-180, 181, 5), crs=ccrs.PlateCarree())
    ax.set_yticks(np.arange(-90, 91, 5), crs=ccrs.PlateCarree())
    ax.set_facecolor("gray")
    plt.title(title)
    plt.clim(clim[0], clim[1])
    cbar = plt.colorbar(cs, ticks=np.linspace(clim[0], clim[1], 6), orientation='horizontal', shrink=0.5)
#     cbar.set_label("Slope")
#     ax.add_patch(patches.Rectangle((-65, -25), 20, 20, linewidth=2, edgecolor='r', facecolor='none'))
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS)
    ax.set_extent(extent)
    plt.show()

In [11]:
def plot_data_anom(dm, ln, lt, extent=[-180, 180, -90, 90], title=''):
    ds = dm.copy()
    fig = plt.figure(figsize=(10,10))
    ax = plt.axes(projection=ccrs.PlateCarree())
    cs = plt.contourf(ln, lt, ds, levels=np.linspace(-100000, 100000, 101), cmap='seismic', extend='both')
    ax.set_xticks(np.arange(-180, 181, 5), crs=ccrs.PlateCarree())
    ax.set_yticks(np.arange(-90, 91, 5), crs=ccrs.PlateCarree())
    plt.title(title)
    plt.clim(-100000, 100000)
    cbar = plt.colorbar(cs, ticks=np.linspace(-100000, 100000, 7), orientation='horizontal', shrink=0.5)
#     cbar.set_label("VPD anomaly (Pa)")
#     ax.add_patch(patches.Rectangle((-85, -35), 55, 30, linewidth=2, edgecolor='b', facecolor='none'))
#     ax.add_patch(patches.Rectangle((-64, -17), 14, 12, linewidth=2, edgecolor='k', facecolor='none'))
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS)
    ax.set_extent(extent)
    plt.show()

In [12]:
def plot_data_std_anom(dm, ln, lt, extent=[-180, 180, -90, 90], cmap='seismic', title=''):
    ds = dm.copy()
    fig = plt.figure(figsize=(10,10))
    ax = plt.axes(projection=ccrs.PlateCarree())
    cs = plt.contourf(ln, lt, ds, levels=np.linspace(-3, 3, 121), cmap=cmap, extend='both')
#     cs2 = plt.contour(cs, levels=[-3, -2, -1, 1, 2, 3], colors='gray')
#     ax.clabel(cs2, cs2.levels, inline=True)
    ax.set_xticks(np.arange(-180, 181, 5), crs=ccrs.PlateCarree())
    ax.set_yticks(np.arange(-90, 91, 5), crs=ccrs.PlateCarree())
    plt.title(title)
    plt.clim(-3, 3)
    cbar = plt.colorbar(cs, ticks=np.linspace(-3, 3, 7), orientation='horizontal', shrink=0.5)
    cbar.set_label("\sigma")
#     cbar.add_lines(cs2)
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS)
    ax.set_extent(extent)
    plt.show()

In [13]:
def plot_correlation_map(dm, ln, lt, extent=[-180, 180, -90, 90], title=''):
    ds = dm.copy()
    fig = plt.figure(figsize=(10,10))
    ax = plt.axes(projection=ccrs.PlateCarree())
    cs = plt.contourf(ln, lt, ds, levels=np.linspace(0, 1, 101), cmap='gnuplot2')
    ax.set_xticks(np.arange(-180, 181, 5), crs=ccrs.PlateCarree())
    ax.set_yticks(np.arange(-90, 91, 5), crs=ccrs.PlateCarree())
    plt.title(title)
    plt.clim(0, 1)
    cbar = plt.colorbar(cs, ticks=np.linspace(0, 1, 11), orientation='horizontal', shrink=0.5)
    cbar.set_label("r")
    ax.add_patch(patches.Rectangle((-65, -17), 15, 12, linewidth=2, edgecolor='w', facecolor='none'))
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS)
    ax.set_extent(extent)
    plt.show()

All data preprocessing: Subtract global mean, then calculate mean, \sigma, and their anomalies

In [14]:
def preprocess_data(x, nMov=False, standardized=True, vpd=True, yr2000=True, anomalies=True, monthly=False, extreme=False, pct=95):
    #determine domain size
#     if x.shape[1] == 121:
#         lat_array = np.arange(-5, -35.1, -0.25)
# #     elif x.shape[2] == 281:
# #         lat_array = np.arange(60, 9.9, -0.25)
#     elif x.shape[1] == 201:
#         lat_array = np.arange(15, -35.1, -0.25)
# #     elif x.shape[2] == 70: 
# #         lat_array = np.arange(59.5, 10, -1)
#     else:
#         if x.shape[1] == 51:
#             lat_array = np.arange(15, -35.1, -1)
#         else:
#             lat_array = np.arange(14.5, -35, -1)
    
    #cos(lat) weighted mean
#     x_cos = x * np.expand_dims(np.cos(lat_array * np.pi / 180) ** 2, axis=(0,2))
    
    #nMov day avg
    if nMov:
        x_xr = xr.DataArray(x, dims=['time', 'latitude', 'longitude'])
        x_nMov_avg = x_xr.rolling(time=5, min_periods=1, center=True).mean().to_numpy()
        x = x_nMov_avg

#     Subtract the global spatial mean from each day. i.e. subtract Jan 1 1979 spatial mean from Jan 1 1979 values
    if not vpd:
        x_global_mean_removed = (x.T - np.mean(x, axis=(1, 2))).T
        x = x_global_mean_removed
    else:
        x_global_mean_removed = x
    
    if monthly:
        x_reshaped = x_global_mean_removed.reshape(int(x.shape[0] / 12), 12, x.shape[1], x.shape[2])
    else:
        x_reshaped = x_global_mean_removed.reshape(int(x.shape[0] / 365), 365, x.shape[1], x.shape[2])
#     x_reshaped = x
    
    if not anomalies:
        return x_reshaped
    
    #Calculate mean, std, anomalies, and std anomalies based on climatology. Two options: 1979-2000, 1979-2014
    if yr2000:
        if extreme: 
            x_mean = np.percentile(x_reshaped[:22], pct, axis=0)
        else:
            x_mean = np.mean(x_reshaped[:22], axis=0)
        x_std = np.std(x_reshaped[:22], axis=0)
    else:
        if extreme: 
            x_mean = np.percentile(x_reshaped[:36], pct, axis=0)
        else:
            x_mean = np.mean(x_reshaped[:36], axis=0)
        x_std = np.std(x_reshaped[:36], axis=0)
        
    if standardized:
        x_std_anom_reshaped = (x_reshaped - x_mean) / x_std
        x_std_anom = x_std_anom_reshaped.reshape(x.shape[0], x.shape[1], x.shape[2])
        return x_std_anom
    else:
        x_anom_reshaped = x_reshaped - x_mean
        x_anom = x_anom_reshaped.reshape(x.shape[0], x.shape[1], x.shape[2])
        return x_anom

In [15]:
djf_indices = np.concatenate([np.arange(59), np.arange(334, 365)])
mam_indices = np.arange(59, 151)
jja_indices = np.arange(151, 243)
son_indices = np.arange(243, 334)
jjaso_indices = np.arange(151, 304)
jjason_indices = np.arange(151, 334)
aso_indices = np.arange(212, 304)
jfmamnd_indices = np.concatenate([np.arange(151), np.arange(304, 365)])

In [14]:
# t2_anom_sa = preprocess_data(t2_sa, standardized=False, vpd=True)
# d2_anom_sa = preprocess_data(d2_sa, standardized=False, vpd=True)

# t2_std_anom_sa = preprocess_data(t2_sa, standardized=True, vpd=True)
# d2_std_anom_sa = preprocess_data(d2_sa, standardized=True, vpd=True)

In [ ]:
# z250_anom_sa = preprocess_data(z250_sa, standardized=False)
# z250_std_anom_sa = preprocess_data(z250_sa)

# z500_anom_sa = preprocess_data(z500_sa, standardized=False)
z500_std_anom_sa = preprocess_data(z500_sa)

# z700_anom_sa = preprocess_data(z700_sa, standardized=False)
# z700_std_anom_sa = preprocess_data(z700_sa)

# z850_anom_sa = preprocess_data(z850_sa, standardized=False)
# z850_std_anom_sa = preprocess_data(z850_sa)

# u250_anom_sa = preprocess_data(u250_sa, standardized=False)
# u250_std_anom_sa = preprocess_data(u250_sa)

# u500_anom_sa = preprocess_data(u500_sa, standardized=False)
# u500_std_anom_sa = preprocess_data(u500_sa)

# u700_anom_sa = preprocess_data(u700_sa, standardized=False)
# u700_std_anom_sa = preprocess_data(u700_sa)

# u850_anom_sa = preprocess_data(u850_sa, standardized=False)
# u850_std_anom_sa = preprocess_data(u850_sa)

# v250_anom_sa = preprocess_data(v250_sa, standardized=False)
# v250_std_anom_sa = preprocess_data(v250_sa)

# v500_anom_sa = preprocess_data(v500_sa, standardized=False)
# v500_std_anom_sa = preprocess_data(v500_sa)

# v700_anom_sa = preprocess_data(v700_sa, standardized=False)
# v700_std_anom_sa = preprocess_data(v700_sa)

# v850_anom_sa = preprocess_data(v850_sa, standardized=False)
# v850_std_anom_sa = preprocess_data(v850_sa)

# r250_anom_sa = preprocess_data(r250_sa, standardized=False)
# r250_std_anom_sa = preprocess_data(r250_sa)

# r500_anom_sa = preprocess_data(r500_sa, standardized=False)
r500_std_anom_sa = preprocess_data(r500_sa)

# r700_anom_sa = preprocess_data(r700_sa, standardized=False)
# r700_std_anom_sa = preprocess_data(r700_sa)

# r850_anom_sa = preprocess_data(r850_sa, standardized=False)
# r850_std_anom_sa = preprocess_data(r850_sa)

In [22]:
vpd_mean = np.mean(vpd_sa.reshape(-1, 365, 201, 221)[:22], axis=(0))
es_mean = np.mean(es_sa.reshape(-1, 365, 201, 221)[:22], axis=(0))
ea_mean = np.mean(ea_sa.reshape(-1, 365, 201, 221)[:22], axis=(0))

In [ ]:
# old_vpd_mean = np.mean(vpd_sa.reshape(-1, 365, 201, 221)[:22,:], axis=(0))
# old_es_mean = np.mean(es_sa.reshape(-1, 365, 201, 221)[:22,:], axis=(0))
# old_ea_mean = np.mean(ea_sa.reshape(-1, 365, 201, 221)[:22,:], axis=(0))

In [23]:
vpd_percentile = np.percentile(vpd_sa.reshape(-1, 365, 201, 221)[:22], 95, axis=(0))
es_percentile = np.percentile(es_sa.reshape(-1, 365, 201, 221)[:22], 95, axis=(0))
ea_percentile = np.percentile(ea_sa.reshape(-1, 365, 201, 221)[:22], 5, axis=(0))

In [ ]:
# old_vpd_anom_sa = vpd_sa.reshape(-1, 365, 201, 221)[:,jjaso_indices] - old_vpd_mean
# old_es_anom_sa = es_sa.reshape(-1, 365, 201, 221)[:,jjaso_indices] - old_es_mean
# old_ea_anom_sa = ea_sa.reshape(-1, 365, 201, 221)[:,jjaso_indices] - old_ea_mean

In [24]:
vpd_anom_sa = vpd_sa.reshape(-1, 365, 201, 221) - vpd_mean
es_anom_sa = es_sa.reshape(-1, 365, 201, 221) - es_mean
ea_anom_sa = ea_sa.reshape(-1, 365, 201, 221) - ea_mean

In [ ]:
extreme_vpd_anom_sa = vpd_sa.reshape(-1, 365, 201, 221) - vpd_percentile
extreme_es_anom_sa = es_sa.reshape(-1, 365, 201, 221) - es_percentile
extreme_ea_anom_sa = ea_sa.reshape(-1, 365, 201, 221) - ea_percentile

In [20]:
# precip_anom_sa = preprocess_data(precip_sa, standardized=False, vpd=True)

Detrending functions

In [25]:
def predict_data_linear_regression(x, noncenter=True):
    detrended = np.empty((x.shape[0], x.shape[1], x.shape[2]))
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            if np.isnan(np.mean(x[:,i,j])):
                continue
            reg = LinearRegression(fit_intercept=True).fit(np.arange(x.shape[0]).reshape(-1, 1), x[:,i,j])
            detrended[:,i,j] = reg.intercept_ + reg.coef_ * np.arange(x.shape[0])
            if noncenter:
                detrended[:,i,j] -= reg.intercept_
    return detrended

In [26]:
def linear_regression(x, center=True):
    reg = LinearRegression(fit_intercept=True).fit(np.arange(x.shape[0]).reshape(-1, 1), x)
    if not center:
        return reg.coef_ * np.arange(x.shape[0])
    return reg.intercept_ + reg.coef_ * np.arange(x.shape[0])

In [27]:
def linear_regression_slope_map(x):
    slopes = np.empty((x.shape[-2], x.shape[-1]))
    significance = np.empty((x.shape[-2], x.shape[-1]))
    for i in range(x.shape[-2]):
        for j in range(x.shape[-1]):
            X = sm.add_constant(np.arange(x.shape[0]))
            model = sm.OLS(x[:,i,j], X)
            result = model.fit()
            slopes[i,j] = result.params[1]
            significance[i,j] = result.pvalues[1] < 0.05 #Use 5% level for significance
    return slopes, significance

In [21]:
# t2_anom_sa_detrended = t2_anom_sa - predict_data_linear_regression(t2_anom_sa, False)

In [20]:
# d2_anom_sa_detrended = d2_anom_sa - predict_data_linear_regression(d2_anom_sa, False)

In [36]:
#Detrend VPD data with linear regression
vpd_anom_sa_detrended = vpd_anom_sa.reshape(45*365, 201, 221) - predict_data_linear_regression(vpd_anom_sa.reshape(45*365, 201, 221), False)
# vpd_std_anom_sa_detrended = vpd_std_anom_sa - predict_data_linear_regression(vpd_std_anom_sa, False)

In [37]:
es_anom_sa_detrended = es_anom_sa.reshape(45*365, 201, 221) - predict_data_linear_regression(es_anom_sa.reshape(45*365, 201, 221), False)
ea_anom_sa_detrended = ea_anom_sa.reshape(45*365, 201, 221) - predict_data_linear_regression(ea_anom_sa.reshape(45*365, 201, 221), False)

# es_std_anom_sa_detrended = es_std_anom_sa - predict_data_linear_regression(es_std_anom_sa, False)
# ea_std_anom_sa_detrended = ea_std_anom_sa - predict_data_linear_regression(ea_std_anom_sa, False)

In [27]:
# precip_anom_sa_detrended = precip_anom_sa - predict_data_linear_regression(precip_anom_sa, False)

The actual Flow Analog function and performing it on the relevant data

In [38]:
# flow_analogue flow analogue search
# [DIST, IDOY, IYR, ID] = flowanalogue(X, Lon, Lat, indYrRef, nFA) 
# returns analogue day index and distance between analogue and observation.
# Lon and Lat are the longitudes and latitudes in a vector; X is the input
# circulation pattern variable array (e.g. geopotential) with the size of
# [N(lon), N(lat), N(time)]; indYrRef is the reference
# year index for searching analogues, e.g. 1:32; nFA is the number of flow
# analogue returned to output. Ouput analogue indices include ID, IDOY, 
# and IYR: IDOY and IYR are index of DOY (day of year) and year, and ID  
# equals IDOY + (IYR-1)*N(day-of-year). All output has the same size of 
# [N(day-of-year), N(year), nFA, N(ensemble)].
#  
# [DIST, IDOY, IYR, ID] = flowanalogue(..., OPTIONAL_ARGUMENT, ARG_VALUE)
# Optional arguments include:
#     nd: half window size for analogue search, window size is nd*2+1.
#       Default is 30;
#
#    distFunc: method for calculating distance, include "Pearson", 
#      "Spearman", and"Euclidean" (default)
#
#    optMov: option to perform moving average, true (default) or false. 
#    nMov: moving average size. Default is 5.
#
#    nEns: flow analogue ensemble size; for each ensemble, analogue is 
#      search every nEns days. Default is 5.
#
#    optCrop: option to crop certain region, true or false (default).
#    lonb: bounding box for longitude, ie. [-160,-80]
#    latb: bounding range for latitude, ie. [20, 60]
#
#    optEof: option to perform EOF and then use reconstructed data as 
#      input, true (default) or false.
#    thEof: cutoff explained variance (%) for EOF. Default is 90.

#    indYrRef: index of years to search for analogues, default is 1:N(year)

#    optNormalize: option to normalize data, include "none", "anomaly", 
#      "standardized anomaly", "smooth anomaly", and "smooth standardized 
#      anomaly" (default).
#    indClm: index of years for calculating climatology. Default is the 
#      same as the reference period (indYrRef)
#    nSmClm: window size for calculating smooth climatology. Default is 31.


#  Yizhou Zhuang (zhuangyz@atmos.ucla.edu)
#  References:
#    Zhuang et al. 2021 (PNAS)
#    Yiou et al. 2007 (GRL)
#

def flowanalogue(X, lon, lat, nFA, distfunc='euclidean', nVar=1, bootstrap=False): 
    
    nlon = X.shape[2]
    nlat = X.shape[1]
    assert nlon == len(lon), 'Size mismatch between input X and lon'
    assert nlat == len(lat), 'Size mismatch between input X and lat'

#     nDoy = len(X.shape[2] / 365)  #number of days in year, must be <= 365
#     assert nDoy <= 365, 'Days of a year exceed 365 days'
    nDoy = 365

    nYr = X.shape[0] // 365   #number of years
    
    #default parameters
    nd = 30
    optMov = False
    nMov = 5
    nEns = 5
    optCrop = False
    lonb = [-85, -30] 
    latb = [-35, -5]
    optEof = False
    thEof = 90 
    optNormalize = 'smooth standard anomaly'
    nSmClm = 31
    indYrRef = np.arange(22) #change this to only take certain years as input. 22 for 2000, 32 for 2010. nYr for whole domain
    indClm = indYrRef
    
#     #parameters with default value
#     nd = 30  #calendar window for analogue search is 2*nd+1 days centered on current day

#     optMov = True #option to perform moving average on input data X
#     nMov = 5 #moving average length

#     nEns = 5 #search analogue every nEns days, creating nEns ensembles

#     optCrop = False #option to use certain region for flow analogue search
#     lonb = bounds(lon) #bounding range for longitude, ie. [-160, -80]
#     latb = bounds(lat) #bounding range for latitude, ie. [20, 60]

#     optEof = True  #perform EOF on data X and then use reconstructed X as input
#     thEof = 90 #cutoff explained variance for EOF

#     #indYrRef = 1 : 32   #use 1979-2010 (input is 1979-2020)
#     indYrRef = 1 : nYr  #use all years

#     optNormalize = "smooth standardized anomaly"
#     indClm = indYrRef  #year index for calculating climatology; default is the same as reference period
#     nSmClm = 31 #window size for calculating smooth climatology

#     distFunc = "Euclidean", "Pearson", "Spearman"
#     if nVar == 1:
#         X = xr.DataArray(X, dims=['time', 'latitude', 'longitude'])
#     else:
#         X = xr.DataArray(X, dims=['time', 'latitude', 'longitude', 'var'])
    #Optional pre-processing
    #perform moving average for input X
    if optMov:
        X = X.rolling(time=nMov, min_periods=1, center=True).mean() #may still want to set endpoints to NAN
    #crop certain region of X for searching flow analogue
    if optCrop:
        X = X.where(X['longitude'] >= lonb[0]).dropna('longitude')
        X = X.where(X['longitude'] <= lonb[1]).dropna('longitude')
        X = X.where(X['latitude'] >= latb[0]).dropna('latitude')
        X = X.where(X['latitude'] <= latb[1]).dropna('latitude')
        nlon = X.shape[1]
        nlat = X.shape[2]
    #Option to normalize X data
    if optNormalize != "none":
        #from http://nicolasfauchereau.github.io/climatecode/posts/eof-analysis-with-scikit-learn/
#         init_scaler = preprocessing.StandardScaler()
#         X = X.stack(space=("latitude", "longitude"))
#         scaler = init_scaler.fit(X)
#         X = scaler.transform(X)
            
        X = (X - np.nanmean(X)) / np.nanstd(X)
                
        assert np.round(np.nanmean(X)) == 0.0
        assert np.round(np.nanstd(X)) == 1.0
        
#         X = X.to_numpy()
    #perform EOF reconstruction
    ### DOUBLE CHECK THIS SECTION LATER
    if optEof: #ignore EOF parts for now
        yq, tmp = np.meshgrid(lat, lon)
        w = np.sqrt(np.cos(yq * np.pi/180)).flatten()  #grid weighting
        
        skpca = PCA()
        skpca.fit((X * w))
        ipc = np.where(skpca.explained_variance_ratio_.cumsum() >= thEof / 100)[0][0]
        print(ipc)
        P = skpca.transform((X * w))
        P = P[:,:ipc]
        E = skpca.components_
        E = E[:ipc,:].T
        print(X.shape, E.shape, P.shape, w.shape)
        X = np.matmul(E, P.T).T / w

        nCCA = P.shape[1] #use EOF number to repsent CCA number
        if nFA < nCCA:
            print('Warning! Analogue number is less than EOF number.')
    else:
        nCCA = nFA
    #Data preparation and output initialization
    #prepare input array X for flow analogue searching
    X0 = np.reshape(X, (nYr*nDoy, nlon*nlat*nVar))
    X = np.reshape(X, (nYr, nDoy, nlon*nlat*nVar))
    X2 = np.concatenate((np.empty((nd, nlon*nlat*nVar)), X[0,:,:], X[1,:nd,:])) #add nd missing points at beginning
    
    ti = np.arange(nDoy*nYr) #create a temporary time index array
    ti2 = np.concatenate((np.empty(nd)*np.nan, ti[:nDoy+nd]))
    for j in range(1, nYr-1):
        X2 = np.concatenate((X2, X[j-1,-nd:,:], X[j,:,:], X[j+1,:nd,:])) #add nd days on both ends to match window size
        ti2 = np.concatenate((ti2, ti[-nd+nDoy*j:nDoy*(j+1)+nd]))
    X2 = np.concatenate((X2, X[-2,-nd:,:], X[-1,:,:], np.empty((nd, nlon*nlat*nVar))))
    ti2 = np.concatenate((ti2, ti[-nd+nDoy*(nYr-1):], np.empty(nd)*np.nan))
    
    X = np.reshape(X2, (nYr, (nDoy+2*nd), nlon*nlat*nVar))
    ti = np.reshape(ti2, (nYr,(nDoy+2*nd)))
#     X = np.concat(np.concat(np.empty(nlon*nlat,nd,1), X[:,nDoy-nd+1:,1:end-1], axis=2), 
#         X, np.concat(X[:,1:nd,2:], np.empty(nlon*nlat,nd,1), axis=2), axis=1)
#     if optEof:
#         nEof = size(P, 2)
#         P = reshape(P.T, nEof, nDoy, nYr); P0 = reshape(P, nEof, []);
#         P = np.concat(2, np.concat(3, nan(nEof,nd,1), P(:,nDoy-nd+1:end,1:end-1)), ...
#             P, np.concat(3, P[:,1:nd,2:end], nan(nEof,nd,1)))
    #create a temporary time index array
#     ti = np.reshape(np.arange(nDoy*nYr), (nYr, nDoy))
#     if nDoy == 365:
#         ti = [[np.empty(nd), ti[nDoy-nd+1:,:ti.shape[1]-1]], ti, [ti[:nd,1:], np.empty(nd)]]
#     else:
#         ti = [np.empty((nd,nYr)), ti, np.empty((nd,nYr))]
#     ti = np.concatenate((np.empty((nYr,nd)), ti, np.empty((nYr,nd))))
    #initialize distance array
    if bootstrap:
        dist = np.empty((nFA*2, nYr, nDoy, nEns))
        b_cca = np.zeros((nFA*2, nYr, nDoy, nEns))
    else:
        dist = np.empty((nFA, nYr, nDoy, nEns))
        b_cca = np.zeros((nFA, nYr, nDoy, nEns))
    i_d = dist 
    iDoy = dist
    iYr = dist
    

    #search flow analogue for each calendar day
    for i in range(nDoy):
        #temporary time index array for analogue ID assignment
        ti1 = ti[:,i:i+2*nd+1].flatten()
        #calculate distance between current day and all possible analogue days
        if distfunc == "pearson":
            X1 = np.reshape(X[:,i:i+2*nd+1,:], (nYr*(2*nd+1), nlon*nlat*nVar))
            dist0 = 1 - np.corrcoef(X1.T, X[:,i+nd+1,:].T, rowvar=False)[X1.shape[0]:,:X1.shape[0]].T
        if distfunc == "spearman":
            X1 = np.reshape(X[:,i:i+2*nd+1,:], (nYr*(2*nd+1), nlon*nlat*nVar))
            dist0 = 1 - stats.spearmanr(X1.T, X[:,i+nd+1,:].T)[0][X1.shape[0]:,:X1.shape[0]].T
        if distfunc == "euclidean":
            if optEof:
                dist0 = cdist(np.reshape(P[:,i:i+2*nd+1,:], (nYr*(2*nd+1), nEof)), P[:,i+nd+1,:])  
            else:
                dist0 = cdist(np.reshape(X[:,i:i+2*nd+1,:], (nYr*(2*nd+1), nlon*nlat*nVar)), X[:,i+nd+1,:])
        #sort distance to current calendar day of all years to search analogue
        dist0_sorted = np.sort(dist0, axis=0)
        dist0_ind = np.argsort(dist0, axis=0)
        #get index of year
        dist0_indYr = np.floor(dist0_ind / (2*nd+1))
        #current year and years not in the reference period will not be considered
        FAindYr = (dist0_indYr != np.tile(np.arange(nYr),((2*nd+1)*nYr,1))) & (np.isin(dist0_indYr, indYrRef)) 
        #loop for all years
        for j in range(nYr):
            dist0 = dist0_sorted[FAindYr[:,j],j]
            id0 = ti1[dist0_ind[FAindYr[:,j],j]]
            if np.all(np.isnan(dist0)):
                dist[:,j,i,:] = np.nan
                i_d[:,j,i,:] = np.nan
                b_cca[:,j,i,:] = np.nan
            else:
                for k in range(nEns):
                    ind = np.nonzero((id0 % nEns == k).T.flatten())[0]
                    if bootstrap:
                        dist[:,j,i,k] = dist0[ind[:nFA*2]] #Modified such that we take first nFA*2 elements of ind and randomly take nFA numbers. Bootstrap this whole process >100 times. Don't take the average of the three distance functions when inputting samples for boostrap.
                        i_d[:,j,i,k] = id0[ind[:nFA*2]]
                    else:    
                        dist[:,j,i,k] = dist0[ind[:nFA]]
                        i_d[:,j,i,k] = id0[ind[:nFA]]
                    #CCA
                    nCCA1 = nCCA
                    if optEof:
                        while np.linalg.matrix_rank(X0[:,i_d[:nCCA1,j,i,k]]) < nCCA:
                            nCCA1 = nCCA1 + 1
                    if bootstrap:
                        nCCA1 *= 2
                    B = LinearRegression(fit_intercept=False).fit(X0[i_d[:nCCA1,j,i,k].astype(int),:].T, X0[(j)*nDoy+i,:]).coef_
                    b_cca[:nCCA1,j,i,k] = B
        iDoy = i_d % nDoy
        iYr = np.floor(i_d/nDoy) 
    return dist, iDoy, iYr, i_d, b_cca

In [ ]:
zr500_std_anom_sa = np.moveaxis(np.array([z500_std_anom_sa, r500_std_anom_sa]), 0, -1)
zr500_std_anom_sa_narrowed = zr500_std_anom_sa #No longer removing NH circulation data for training

In [ ]:
zr500_index_time_euclidean_sa, zr500_b_cca_euclidean_sa = flowanalogue(zr500_std_anom_sa_narrowed, np.arange(275, 330.1, 0.25), np.arange(15, -35.1, -0.25), 10, 'euclidean', 2)[3:5]
zr500_index_time_pearson_sa, zr500_b_cca_pearson_sa = flowanalogue(zr500_std_anom_sa_narrowed, np.arange(275, 330.1, 0.25), np.arange(15, -35.1, -0.25), 10, 'pearson', 2)[3:5]
zr500_index_time_spearman_sa, zr500_b_cca_spearman_sa = flowanalogue(zr500_std_anom_sa_narrowed, np.arange(275, 330.1, 0.25), np.arange(15, -35.1, -0.25), 10, 'spearman', 2)[3:5]

In [ ]:
np.save(prefix2 + "SA_output/zr500_index_time_euclidean_sa", zr500_index_time_euclidean_sa)
np.save(prefix2 + "SA_output/zr500_b_cca_euclidean_sa", zr500_b_cca_euclidean_sa)

np.save(prefix2 + "SA_output/zr500_index_time_pearson_sa", zr500_index_time_pearson_sa)
np.save(prefix2 + "SA_output/zr500_b_cca_pearson_sa", zr500_b_cca_pearson_sa)

np.save(prefix2 + "SA_output/zr500_index_time_spearman_sa", zr500_index_time_spearman_sa)
np.save(prefix2 + "SA_output/zr500_b_cca_spearman_sa", zr500_b_cca_spearman_sa)

In [39]:
zr500_b_cca_euclidean_sa = np.load(prefix2 + "SA_output/zr500_b_cca_euclidean_sa.npy")
zr500_index_time_euclidean_sa = np.load(prefix2 + "SA_output/zr500_index_time_euclidean_sa.npy")

zr500_b_cca_pearson_sa = np.load(prefix2 + "SA_output/zr500_b_cca_pearson_sa.npy")
zr500_index_time_pearson_sa = np.load(prefix2 + "SA_output/zr500_index_time_pearson_sa.npy")

zr500_b_cca_spearman_sa = np.load(prefix2 + "SA_output/zr500_b_cca_spearman_sa.npy")
zr500_index_time_spearman_sa = np.load(prefix2 + "SA_output/zr500_index_time_spearman_sa.npy")

In [40]:
def calculate_analogue_array_vpd_zr500_sa(x, yr, Doy): #Specify in 4-digit year, day number of year. i.e. calculate_analogue_array(2020, 228)
    
    euclidean = np.mean(np.sum(((x[zr500_index_time_euclidean_sa[:,yr-1979,Doy,:].reshape(50).astype(int),:,:] * np.expand_dims(zr500_b_cca_euclidean_sa[:,yr-1979,Doy,:].reshape(50), axis=(1, 2))).reshape(10, 5, 201, 221)), axis=0), axis=0)
    pearson = np.mean(np.sum(((x[zr500_index_time_pearson_sa[:,yr-1979,Doy,:].reshape(50).astype(int),:,:] * np.expand_dims(zr500_b_cca_pearson_sa[:,yr-1979,Doy,:].reshape(50), axis=(1, 2))).reshape(10, 5, 201, 221)), axis=0), axis=0)
    spearman = np.mean(np.sum(((x[zr500_index_time_spearman_sa[:,yr-1979,Doy,:].reshape(50).astype(int),:,:] * np.expand_dims(zr500_b_cca_spearman_sa[:,yr-1979,Doy,:].reshape(50), axis=(1, 2))).reshape(10, 5, 201, 221)), axis=0), axis=0)
    return np.mean([euclidean, pearson, spearman], axis=0)

In [41]:
def calculate_analogue_array_vpd_sa(x, yr, Doy, euclidean_index, pearson_index, spearman_index, euclidean_b, pearson_b, spearman_b): #Specify in 4-digit year, day number of year. i.e. calculate_analogue_array(2020, 228)
    
    euclidean = np.mean(np.sum(((x[euclidean_index[:,yr-1979,Doy,:].reshape(50).astype(int),:,:] * np.expand_dims(euclidean_b[:,yr-1979,Doy,:].reshape(50), axis=(1, 2))).reshape(10, 5, 201, 221)), axis=0), axis=0)
    pearson = np.mean(np.sum(((x[pearson_index[:,yr-1979,Doy,:].reshape(50).astype(int),:,:] * np.expand_dims(pearson_b[:,yr-1979,Doy,:].reshape(50), axis=(1, 2))).reshape(10, 5, 201, 221)), axis=0), axis=0)
    spearman = np.mean(np.sum(((x[spearman_index[:,yr-1979,Doy,:].reshape(50).astype(int),:,:] * np.expand_dims(spearman_b[:,yr-1979,Doy,:].reshape(50), axis=(1, 2))).reshape(10, 5, 201, 221)), axis=0), axis=0)
    return np.mean([euclidean, pearson, spearman], axis=0)

In [42]:
def calculate_analogue_array_precip_zr500_sa(x, yr, Doy): #Specify in 4-digit year, day number of year. i.e. calculate_analogue_array(2020, 228)
    
    euclidean = np.mean(np.sum(((x[zr500_index_time_euclidean_sa[:,yr-1979,Doy,:].reshape(50).astype(int),:,:] * np.expand_dims(zr500_b_cca_euclidean_sa[:,yr-1979,Doy,:].reshape(50), axis=(1, 2))).reshape(10, 5, 100, 110)), axis=0), axis=0)
    pearson = np.mean(np.sum(((x[zr500_index_time_pearson_sa[:,yr-1979,Doy,:].reshape(50).astype(int),:,:] * np.expand_dims(zr500_b_cca_pearson_sa[:,yr-1979,Doy,:].reshape(50), axis=(1, 2))).reshape(10, 5, 100, 110)), axis=0), axis=0)
    spearman = np.mean(np.sum(((x[zr500_index_time_spearman_sa[:,yr-1979,Doy,:].reshape(50).astype(int),:,:] * np.expand_dims(zr500_b_cca_spearman_sa[:,yr-1979,Doy,:].reshape(50), axis=(1, 2))).reshape(10, 5, 100, 110)), axis=0), axis=0)
    return np.mean([euclidean, pearson, spearman], axis=0)

In [43]:
vpd_anom_analogue_zr500_sa = np.empty((45, 365, 201, 221))
es_anom_analogue_zr500_sa = np.empty((45, 365, 201, 221))
ea_anom_analogue_zr500_sa = np.empty((45, 365, 201, 221))

for yr in range(1979, 2024):
    for dy in range(365):
        if dy == 364 and yr == 2023:
            continue
        vpd_anom_analogue_zr500_sa[yr-1979,dy,:,:] = calculate_analogue_array_vpd_zr500_sa(vpd_anom_sa_detrended, yr, dy)
        es_anom_analogue_zr500_sa[yr-1979,dy,:,:] = calculate_analogue_array_vpd_zr500_sa(es_anom_sa_detrended, yr, dy)
        ea_anom_analogue_zr500_sa[yr-1979,dy,:,:] = calculate_analogue_array_vpd_zr500_sa(ea_anom_sa_detrended, yr, dy)
    print(yr)

1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [44]:
np.save(prefix2 + "SA_output/vpd_anom_analogue_zr500_sa", vpd_anom_analogue_zr500_sa)
np.save(prefix2 + "SA_output/es_anom_analogue_zr500_sa", es_anom_analogue_zr500_sa)
np.save(prefix2 + "SA_output/ea_anom_analogue_zr500_sa", ea_anom_analogue_zr500_sa)

In [32]:
vpd_anom_analogue_zr500_sa = np.load(prefix2 + "SA_output/vpd_anom_analogue_zr500_sa.npy")
es_anom_analogue_zr500_sa = np.load(prefix2 + "SA_output/es_anom_analogue_zr500_sa.npy")
ea_anom_analogue_zr500_sa = np.load(prefix2 + "SA_output/ea_anom_analogue_zr500_sa.npy")

In [27]:
t2_anom_analogue_zr500_sa = np.empty((45, 365, 201, 221))

for yr in range(1979, 2024):
    for dy in range(365):
        if dy == 364 and yr == 2023:
            continue
        t2_anom_analogue_zr500_sa[yr-1979,dy,:,:] = calculate_analogue_array_vpd_zr500_sa(t2_anom_sa_detrended, yr, dy)
    print(yr)

1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [28]:
np.save(prefix2 + "SA_output/t2_anom_analogue_zr500_sa", t2_anom_analogue_zr500_sa)

In [ ]:
t2_anom_analogue_zr500_sa = np.load(prefix2 + "SA_output/t2_anom_analogue_zr500_sa.npy")

In [21]:
d2_anom_analogue_zr500_sa = np.empty((45, 365, 201, 221))

for yr in range(1979, 2024):
    for dy in range(365):
        if dy == 364 and yr == 2023:
            continue
        d2_anom_analogue_zr500_sa[yr-1979,dy,:,:] = calculate_analogue_array_vpd_zr500_sa(d2_anom_sa_detrended, yr, dy)
    print(yr)

1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [22]:
np.save(prefix2 + "SA_output/d2_anom_analogue_zr500_sa", d2_anom_analogue_zr500_sa)

In [ ]:
d2_anom_analogue_zr500_sa = np.load(prefix2 + "SA_output/d2_anom_analogue_zr500_sa.npy")

In [22]:
precip_anom_analogue_zr500_sa = np.empty((45, 365, 100, 110))

for yr in range(1979, 2024):
    for dy in range(365):
        if dy == 364 and yr == 2023:
            continue
        precip_anom_analogue_zr500_sa[yr-1979,dy,:,:] = calculate_analogue_array_precip_zr500_sa(precip_anom_sa_detrended, yr, dy)
    print(yr)

1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [23]:
np.save(prefix2 + "SA_output/precip_anom_analogue_zr500_sa", precip_anom_analogue_zr500_sa)

In [ ]:
precip_anom_analogue_zr500_sa = np.load(prefix2 + "SA_output/precip_anom_analogue_zr500_sa.npy")

In [31]:
def bootstrap_anthropogenic_day(x, nFA, nR, vr): #nR is number of times to repeat bootstrap process    
    for i in range(nR):
        random_numbers = np.random.randint(0, 20, 10)
        euclidean = np.empty((45, 365, 201, 221))
        pearson = np.empty((45, 365, 201, 221))
        spearman = np.empty((45, 365, 201, 221))
        for yr in range(1979, 2024):
            for dy in range(365):
                if dy == 364 and yr == 2023:
                    continue
                euclidean[yr-1979,dy,:,:] = np.mean(np.sum(((x[zr500_index_time_bootstrap_euclidean_sa[random_numbers,yr-1979,dy,:].reshape(50).astype(int),:,:] * np.expand_dims(zr500_b_cca_bootstrap_euclidean_sa[random_numbers,yr-1979,dy,:].reshape(50), axis=(1, 2))).reshape(10, 5, 201, 221)), axis=0), axis=0)
                pearson[yr-1979,dy,:,:] = np.mean(np.sum(((x[zr500_index_time_bootstrap_pearson_sa[random_numbers,yr-1979,dy,:].reshape(50).astype(int),:,:] * np.expand_dims(zr500_b_cca_bootstrap_pearson_sa[random_numbers,yr-1979,dy,:].reshape(50), axis=(1, 2))).reshape(10, 5, 201, 221)), axis=0), axis=0)
                spearman[yr-1979,dy,:,:] = np.mean(np.sum(((x[zr500_index_time_bootstrap_spearman_sa[random_numbers,yr-1979,dy,:].reshape(50).astype(int),:,:] * np.expand_dims(zr500_b_cca_bootstrap_spearman_sa[random_numbers,yr-1979,dy,:].reshape(50), axis=(1, 2))).reshape(10, 5, 201, 221)), axis=0), axis=0)
                print(i, yr, dy)
    
        np.save(prefix2 + "SA_output/anthropogenic_day_bootstrap_samples/" + vr + "_euclidean_sample_" + str(i), euclidean)
        np.save(prefix2 + "SA_output/anthropogenic_day_bootstrap_samples/" + vr + "_pearson_sample_" + str(i), pearson)
        np.save(prefix2 + "SA_output/anthropogenic_day_bootstrap_samples/" + vr + "_spearman_sample_" + str(i), spearman)

In [40]:
zr500_index_time_bootstrap_euclidean_sa, zr500_b_cca_bootstrap_euclidean_sa = flowanalogue(zr500_std_anom_sa_narrowed, np.arange(275, 330.1, 0.25), np.arange(15, -35.1, -0.25), 10, 'euclidean', 2, True)[3:5]
zr500_index_time_bootstrap_pearson_sa, zr500_b_cca_bootstrap_pearson_sa = flowanalogue(zr500_std_anom_sa_narrowed, np.arange(275, 330.1, 0.25), np.arange(15, -35.1, -0.25), 10, 'pearson', 2, True)[3:5]
zr500_index_time_bootstrap_spearman_sa, zr500_b_cca_bootstrap_spearman_sa = flowanalogue(zr500_std_anom_sa_narrowed, np.arange(275, 330.1, 0.25), np.arange(15, -35.1, -0.25), 10, 'spearman', 2, True)[3:5]

In [41]:
np.save(prefix2 + "SA_output/zr500_index_time_bootstrap_euclidean_sa", zr500_index_time_bootstrap_euclidean_sa)
np.save(prefix2 + "SA_output/zr500_b_cca_bootstrap_euclidean_sa", zr500_b_cca_bootstrap_euclidean_sa)

np.save(prefix2 + "SA_output/zr500_index_time_bootstrap_pearson_sa", zr500_index_time_bootstrap_pearson_sa)
np.save(prefix2 + "SA_output/zr500_b_cca_bootstrap_pearson_sa", zr500_b_cca_bootstrap_pearson_sa)

np.save(prefix2 + "SA_output/zr500_index_time_bootstrap_spearman_sa", zr500_index_time_bootstrap_spearman_sa)
np.save(prefix2 + "SA_output/zr500_b_cca_bootstrap_spearman_sa", zr500_b_cca_bootstrap_spearman_sa)

In [32]:
zr500_b_cca_bootstrap_euclidean_sa = np.load(prefix2 + "SA_output/zr500_b_cca_bootstrap_euclidean_sa.npy")
zr500_index_time_bootstrap_euclidean_sa = np.load(prefix2 + "SA_output/zr500_index_time_bootstrap_euclidean_sa.npy")

zr500_b_cca_bootstrap_pearson_sa = np.load(prefix2 + "SA_output/zr500_b_cca_bootstrap_pearson_sa.npy")
zr500_index_time_bootstrap_pearson_sa = np.load(prefix2 + "SA_output/zr500_index_time_bootstrap_pearson_sa.npy")

zr500_b_cca_bootstrap_spearman_sa = np.load(prefix2 + "SA_output/zr500_b_cca_bootstrap_spearman_sa.npy")
zr500_index_time_bootstrap_spearman_sa = np.load(prefix2 + "SA_output/zr500_index_time_bootstrap_spearman_sa.npy")

In [ ]:
# bootstrap_anthropogenic_day(vpd_anom_sa_detrended, 10, 100, "vpd")
bootstrap_anthropogenic_day(es_anom_sa_detrended, 10, 100, "es")
bootstrap_anthropogenic_day(ea_anom_sa_detrended, 10, 100, "ea")

0 1979 0
0 1979 1
0 1979 2
0 1979 3
0 1979 4
0 1979 5
0 1979 6
0 1979 7
0 1979 8
0 1979 9
0 1979 10
0 1979 11
0 1979 12
0 1979 13
0 1979 14
0 1979 15
0 1979 16
0 1979 17
0 1979 18
0 1979 19
0 1979 20
0 1979 21
0 1979 22
0 1979 23
0 1979 24
0 1979 25
0 1979 26
0 1979 27
0 1979 28
0 1979 29
0 1979 30
0 1979 31
0 1979 32
0 1979 33
0 1979 34
0 1979 35
0 1979 36
0 1979 37
0 1979 38
0 1979 39
0 1979 40
0 1979 41
0 1979 42
0 1979 43
0 1979 44
0 1979 45
0 1979 46
0 1979 47
0 1979 48
0 1979 49
0 1979 50
0 1979 51
0 1979 52
0 1979 53
0 1979 54
0 1979 55
0 1979 56
0 1979 57
0 1979 58
0 1979 59
0 1979 60
0 1979 61
0 1979 62
0 1979 63
0 1979 64
0 1979 65
0 1979 66
0 1979 67
0 1979 68
0 1979 69
0 1979 70
0 1979 71
0 1979 72
0 1979 73
0 1979 74
0 1979 75
0 1979 76
0 1979 77
0 1979 78
0 1979 79
0 1979 80
0 1979 81
0 1979 82
0 1979 83
0 1979 84
0 1979 85
0 1979 86
0 1979 87
0 1979 88
0 1979 89
0 1979 90
0 1979 91
0 1979 92
0 1979 93
0 1979 94
0 1979 95
0 1979 96
0 1979 97
0 1979 98
0 1979 99
0 1979 100

0 1981 49
0 1981 50
0 1981 51
0 1981 52
0 1981 53
0 1981 54
0 1981 55
0 1981 56
0 1981 57
0 1981 58
0 1981 59
0 1981 60
0 1981 61
0 1981 62
0 1981 63
0 1981 64
0 1981 65
0 1981 66
0 1981 67
0 1981 68
0 1981 69
0 1981 70
0 1981 71
0 1981 72
0 1981 73
0 1981 74
0 1981 75
0 1981 76
0 1981 77
0 1981 78
0 1981 79
0 1981 80
0 1981 81
0 1981 82
0 1981 83
0 1981 84
0 1981 85
0 1981 86
0 1981 87
0 1981 88
0 1981 89
0 1981 90
0 1981 91
0 1981 92
0 1981 93
0 1981 94
0 1981 95
0 1981 96
0 1981 97
0 1981 98
0 1981 99
0 1981 100
0 1981 101
0 1981 102
0 1981 103
0 1981 104
0 1981 105
0 1981 106
0 1981 107
0 1981 108
0 1981 109
0 1981 110
0 1981 111
0 1981 112
0 1981 113
0 1981 114
0 1981 115
0 1981 116
0 1981 117
0 1981 118
0 1981 119
0 1981 120
0 1981 121
0 1981 122
0 1981 123
0 1981 124
0 1981 125
0 1981 126
0 1981 127
0 1981 128
0 1981 129
0 1981 130
0 1981 131
0 1981 132
0 1981 133
0 1981 134
0 1981 135
0 1981 136
0 1981 137
0 1981 138
0 1981 139
0 1981 140
0 1981 141
0 1981 142
0 1981 143
0 1981

0 1983 94
0 1983 95
0 1983 96
0 1983 97
0 1983 98
0 1983 99
0 1983 100
0 1983 101
0 1983 102
0 1983 103
0 1983 104
0 1983 105
0 1983 106
0 1983 107
0 1983 108
0 1983 109
0 1983 110
0 1983 111
0 1983 112
0 1983 113
0 1983 114
0 1983 115
0 1983 116
0 1983 117
0 1983 118
0 1983 119
0 1983 120
0 1983 121
0 1983 122
0 1983 123
0 1983 124
0 1983 125
0 1983 126
0 1983 127
0 1983 128
0 1983 129
0 1983 130
0 1983 131
0 1983 132
0 1983 133
0 1983 134
0 1983 135
0 1983 136
0 1983 137
0 1983 138
0 1983 139
0 1983 140
0 1983 141
0 1983 142
0 1983 143
0 1983 144
0 1983 145
0 1983 146
0 1983 147
0 1983 148
0 1983 149
0 1983 150
0 1983 151
0 1983 152
0 1983 153
0 1983 154
0 1983 155
0 1983 156
0 1983 157
0 1983 158
0 1983 159
0 1983 160
0 1983 161
0 1983 162
0 1983 163
0 1983 164
0 1983 165
0 1983 166
0 1983 167
0 1983 168
0 1983 169
0 1983 170
0 1983 171
0 1983 172
0 1983 173
0 1983 174
0 1983 175
0 1983 176
0 1983 177
0 1983 178
0 1983 179
0 1983 180
0 1983 181
0 1983 182
0 1983 183
0 1983 184
0 198

0 1985 136
0 1985 137
0 1985 138
0 1985 139
0 1985 140
0 1985 141
0 1985 142
0 1985 143
0 1985 144
0 1985 145
0 1985 146
0 1985 147
0 1985 148
0 1985 149
0 1985 150
0 1985 151
0 1985 152
0 1985 153
0 1985 154
0 1985 155
0 1985 156
0 1985 157
0 1985 158
0 1985 159
0 1985 160
0 1985 161
0 1985 162
0 1985 163
0 1985 164
0 1985 165
0 1985 166
0 1985 167
0 1985 168
0 1985 169
0 1985 170
0 1985 171
0 1985 172
0 1985 173
0 1985 174
0 1985 175
0 1985 176
0 1985 177
0 1985 178
0 1985 179
0 1985 180
0 1985 181
0 1985 182
0 1985 183
0 1985 184
0 1985 185
0 1985 186
0 1985 187
0 1985 188
0 1985 189
0 1985 190
0 1985 191
0 1985 192
0 1985 193
0 1985 194
0 1985 195
0 1985 196
0 1985 197
0 1985 198
0 1985 199
0 1985 200
0 1985 201
0 1985 202
0 1985 203
0 1985 204
0 1985 205
0 1985 206
0 1985 207
0 1985 208
0 1985 209
0 1985 210
0 1985 211
0 1985 212
0 1985 213
0 1985 214
0 1985 215
0 1985 216
0 1985 217
0 1985 218
0 1985 219
0 1985 220
0 1985 221
0 1985 222
0 1985 223
0 1985 224
0 1985 225
0 1985 226

0 1987 180
0 1987 181
0 1987 182
0 1987 183
0 1987 184
0 1987 185
0 1987 186
0 1987 187
0 1987 188
0 1987 189
0 1987 190
0 1987 191
0 1987 192
0 1987 193
0 1987 194
0 1987 195
0 1987 196
0 1987 197
0 1987 198
0 1987 199
0 1987 200
0 1987 201
0 1987 202
0 1987 203
0 1987 204
0 1987 205
0 1987 206
0 1987 207
0 1987 208
0 1987 209
0 1987 210
0 1987 211
0 1987 212
0 1987 213
0 1987 214
0 1987 215
0 1987 216
0 1987 217
0 1987 218
0 1987 219
0 1987 220
0 1987 221
0 1987 222
0 1987 223
0 1987 224
0 1987 225
0 1987 226
0 1987 227
0 1987 228
0 1987 229
0 1987 230
0 1987 231
0 1987 232
0 1987 233
0 1987 234
0 1987 235
0 1987 236
0 1987 237
0 1987 238
0 1987 239
0 1987 240
0 1987 241
0 1987 242
0 1987 243
0 1987 244
0 1987 245
0 1987 246
0 1987 247
0 1987 248
0 1987 249
0 1987 250
0 1987 251
0 1987 252
0 1987 253
0 1987 254
0 1987 255
0 1987 256
0 1987 257
0 1987 258
0 1987 259
0 1987 260
0 1987 261
0 1987 262
0 1987 263
0 1987 264
0 1987 265
0 1987 266
0 1987 267
0 1987 268
0 1987 269
0 1987 270

...

5 1988 207
5 1988 208
5 1988 209
5 1988 210
5 1988 211
5 1988 212
5 1988 213
5 1988 214
5 1988 215
5 1988 216
5 1988 217
5 1988 218
5 1988 219
5 1988 220
5 1988 221
5 1988 222
5 1988 223
5 1988 224
5 1988 225
5 1988 226
5 1988 227
5 1988 228
5 1988 229
5 1988 230
5 1988 231
5 1988 232
5 1988 233
5 1988 234
5 1988 235
5 1988 236
5 1988 237
5 1988 238
5 1988 239
5 1988 240
5 1988 241
5 1988 242
5 1988 243
5 1988 244
5 1988 245
5 1988 246
5 1988 247
5 1988 248
5 1988 249
5 1988 250
5 1988 251
5 1988 252
5 1988 253
5 1988 254
5 1988 255
5 1988 256
5 1988 257
5 1988 258
5 1988 259
5 1988 260
5 1988 261
5 1988 262
5 1988 263
5 1988 264
5 1988 265
5 1988 266
5 1988 267
5 1988 268
5 1988 269
5 1988 270
5 1988 271
5 1988 272
5 1988 273
5 1988 274
5 1988 275
5 1988 276
5 1988 277
5 1988 278
5 1988 279
5 1988 280
5 1988 281
5 1988 282
5 1988 283
5 1988 284
5 1988 285
5 1988 286
5 1988 287
5 1988 288
5 1988 289
5 1988 290
5 1988 291
5 1988 292
5 1988 293
5 1988 294
5 1988 295
5 1988 296
5 1988 297

5 1990 244
5 1990 245
5 1990 246
5 1990 247
5 1990 248
5 1990 249
5 1990 250
5 1990 251
5 1990 252
5 1990 253
5 1990 254
5 1990 255
5 1990 256
5 1990 257
5 1990 258
5 1990 259
5 1990 260
5 1990 261
5 1990 262
5 1990 263
5 1990 264
5 1990 265
5 1990 266
5 1990 267
5 1990 268
5 1990 269
5 1990 270
5 1990 271
5 1990 272
5 1990 273
5 1990 274
5 1990 275
5 1990 276
5 1990 277
5 1990 278
5 1990 279
5 1990 280
5 1990 281
5 1990 282
5 1990 283
5 1990 284
5 1990 285
5 1990 286
5 1990 287
5 1990 288
5 1990 289
5 1990 290
5 1990 291
5 1990 292
5 1990 293
5 1990 294
5 1990 295
5 1990 296
5 1990 297
5 1990 298
5 1990 299
5 1990 300
5 1990 301
5 1990 302
5 1990 303
5 1990 304
5 1990 305
5 1990 306
5 1990 307
5 1990 308
5 1990 309
5 1990 310
5 1990 311
5 1990 312
5 1990 313
5 1990 314
5 1990 315
5 1990 316
5 1990 317
5 1990 318
5 1990 319
5 1990 320
5 1990 321
5 1990 322
5 1990 323
5 1990 324
5 1990 325
5 1990 326
5 1990 327
5 1990 328
5 1990 329
5 1990 330
5 1990 331
5 1990 332
5 1990 333
5 1990 334

5 1992 283
5 1992 284
5 1992 285
5 1992 286
5 1992 287
5 1992 288
5 1992 289
5 1992 290
5 1992 291
5 1992 292
5 1992 293
5 1992 294
5 1992 295
5 1992 296
5 1992 297
5 1992 298
5 1992 299
5 1992 300
5 1992 301
5 1992 302
5 1992 303
5 1992 304
5 1992 305
5 1992 306
5 1992 307
5 1992 308
5 1992 309
5 1992 310
5 1992 311
5 1992 312
5 1992 313
5 1992 314
5 1992 315
5 1992 316
5 1992 317
5 1992 318
5 1992 319
5 1992 320
5 1992 321
5 1992 322
5 1992 323
5 1992 324
5 1992 325
5 1992 326
5 1992 327
5 1992 328
5 1992 329
5 1992 330
5 1992 331
5 1992 332
5 1992 333
5 1992 334
5 1992 335
5 1992 336
5 1992 337
5 1992 338
5 1992 339
5 1992 340
5 1992 341
5 1992 342
5 1992 343
5 1992 344
5 1992 345
5 1992 346
5 1992 347
5 1992 348
5 1992 349
5 1992 350
5 1992 351
5 1992 352
5 1992 353
5 1992 354
5 1992 355
5 1992 356
5 1992 357
5 1992 358
5 1992 359
5 1992 360
5 1992 361
5 1992 362
5 1992 363
5 1992 364
5 1993 0
5 1993 1
5 1993 2
5 1993 3
5 1993 4
5 1993 5
5 1993 6
5 1993 7
5 1993 8
5 1993 9
5 1993 1

5 1994 321
5 1994 322
5 1994 323
5 1994 324
5 1994 325
5 1994 326
5 1994 327
5 1994 328
5 1994 329
5 1994 330
5 1994 331
5 1994 332
5 1994 333
5 1994 334
5 1994 335
5 1994 336
5 1994 337
5 1994 338
5 1994 339
5 1994 340
5 1994 341
5 1994 342
5 1994 343
5 1994 344
5 1994 345
5 1994 346
5 1994 347
5 1994 348
5 1994 349
5 1994 350
5 1994 351
5 1994 352
5 1994 353
5 1994 354
5 1994 355
5 1994 356
5 1994 357
5 1994 358
5 1994 359
5 1994 360
5 1994 361
5 1994 362
5 1994 363
5 1994 364
5 1995 0
5 1995 1
5 1995 2
5 1995 3
5 1995 4
5 1995 5
5 1995 6
5 1995 7
5 1995 8
5 1995 9
5 1995 10
5 1995 11
5 1995 12
5 1995 13
5 1995 14
5 1995 15
5 1995 16
5 1995 17
5 1995 18
5 1995 19
5 1995 20
5 1995 21
5 1995 22
5 1995 23
5 1995 24
5 1995 25
5 1995 26
5 1995 27
5 1995 28
5 1995 29
5 1995 30
5 1995 31
5 1995 32
5 1995 33
5 1995 34
5 1995 35
5 1995 36
5 1995 37
5 1995 38
5 1995 39
5 1995 40
5 1995 41
5 1995 42
5 1995 43
5 1995 44
5 1995 45
5 1995 46
5 1995 47
5 1995 48
5 1995 49
5 1995 50
5 1995 51
5 1995

5 1996 364
5 1997 0
5 1997 1
5 1997 2
5 1997 3
5 1997 4
5 1997 5
5 1997 6
5 1997 7
5 1997 8
5 1997 9
5 1997 10
5 1997 11
5 1997 12
5 1997 13
5 1997 14
5 1997 15
5 1997 16
5 1997 17
5 1997 18
5 1997 19
5 1997 20
5 1997 21
5 1997 22
5 1997 23
5 1997 24
5 1997 25
5 1997 26
5 1997 27
5 1997 28
5 1997 29
5 1997 30
5 1997 31
5 1997 32
5 1997 33
5 1997 34
5 1997 35
5 1997 36
5 1997 37
5 1997 38
5 1997 39
5 1997 40
5 1997 41
5 1997 42
5 1997 43
5 1997 44
5 1997 45
5 1997 46
5 1997 47
5 1997 48
5 1997 49
5 1997 50
5 1997 51
5 1997 52
5 1997 53
5 1997 54
5 1997 55
5 1997 56
5 1997 57
5 1997 58
5 1997 59
5 1997 60
5 1997 61
5 1997 62
5 1997 63
5 1997 64
5 1997 65
5 1997 66
5 1997 67
5 1997 68
5 1997 69
5 1997 70
5 1997 71
5 1997 72
5 1997 73
5 1997 74
5 1997 75
5 1997 76
5 1997 77
5 1997 78
5 1997 79
5 1997 80
5 1997 81
5 1997 82
5 1997 83
5 1997 84
5 1997 85
5 1997 86
5 1997 87
5 1997 88
5 1997 89
5 1997 90
5 1997 91
5 1997 92
5 1997 93
5 1997 94
5 1997 95
5 1997 96
5 1997 97
5 1997 98
5 1997 99

In [ ]:
# np.save(prefix2 + "SA_output/vpd_anom_analogue_bootstrap_zr500_sa", vpd_anom_analogue_bootstrap_zr500_sa)
# np.save(prefix2 + "SA_output/es_anom_analogue_bootstrap_zr500_sa", es_anom_analogue_bootstrap_zr500_sa)
# np.save(prefix2 + "SA_output/ea_anom_analogue_bootstrap_zr500_sa", ea_anom_analogue_bootstrap_zr500_sa)

In [38]:
djf_indices = np.concatenate([np.arange(59), np.arange(334, 365)])
mam_indices = np.arange(59, 151)
jja_indices = np.arange(151, 243)
son_indices = np.arange(243, 334)
jjaso_indices = np.arange(151, 304)
jjason_indices = np.arange(151, 334)
aso_indices = np.arange(212, 304)